In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pprint, json, pickle, time, copy

In [2]:
def check_double_entries(same_as_dict, verbose=False):
    
    print(len(same_as_dict))
    duplets = set([])

    for subj,aliases in same_as_dict.items():
        for alias in aliases:
            if alias != subj and alias in same_as_dict.keys():
                duplets.add((subj, alias))
                if verbose is True:
                    print(subj)
                    print(alias)
                    print()

    duplets = list(duplets)
    print("Found {} duplets.".format(len(duplets)))
    return duplets

In [25]:
def merge_same_as(same_as_dict):
    print(len(same_as_dict))
    duplets = check_double_entries(same_as_dict)
    
    for duplet in duplets:
        if duplet[0] in same_as_dict and duplet[1] in same_as_dict:
            same_as_dict[duplet[0]].update(same_as_dict[duplet[1]])
            del same_as_dict[duplet[1]]
            
    print(len(same_as_dict))
    return same_as_dict

In [4]:
sparql = SPARQLWrapper("http://localhost:3040/judaicalink/query")
sparql.setQuery("""
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    SELECT DISTINCT ?s ?same ?g
    WHERE { GRAPH ?g {
        ?s owl:sameAs ?same
        
        }}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [6]:
# SECOND VERSION
blacklist = ['http://www.dnb.de/DE/Service/DigitaleDienste/EntityFacts/entityfacts_node.html']
entity_index = set([])
entities = {}
c = 0
for res in results['results']['bindings']:
    s = res['s']['value']
    same = res['same']['value']
    entity_index.add(s)
    entity_index.add(same)
    if s not in blacklist and same not in blacklist:
    
        # check keys
        if s in entities and same not in entities:
            entities[s].add(same)
        elif same in entities and s not in entities:
            entities[same].add(s)
        elif s in entities and same in entities: # merge two entries!
            entities[s] = entities[s].union(entities[same])
            entities[s].add(same)
            del entities[same]
        else: # check values
            added = False
            for key,val in entities.items():
                if s in val and same not in val:
                    entities[key].add(same)
                    added = True
                    break
                elif same in val and s not in val:
                    entities[key].add(s)
                    added = True
                    break
            if added is False:
                entities[s] = set([same])

        c += 1
        if c % 10000 == 0:
            print(c, end=" ")

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 101000 102000 103000 104000 105000 106000 107000 108000 109000 110000 111000 112000 113000 114000 115000 116000 117000 118000 119000 120000 121000 122000 123000 124000 125000 126000 127000 128000 129000 130000 131000 132000 133000 134000 135000 136000 137000 138000 139000 140000 141000 142000 143000 144000 145000 146000 147000 148000 149000 150000 151000 152000 153000 154000 155000 156000 157000 158000 15

In [26]:
merged = merge_same_as_pools(entities)

54362
54362
Found 4860 duplets.
49599


In [27]:
e_index = {k: set([]) for k in entity_index}

In [36]:
# create sets
pools = []
for k,v in merged.items():
    pool = v
    pool.add(k)
    pools.append(pool)
print(len(pools))
pools = sorted(pools, key=lambda x:len(x), reverse=True)

49599


In [38]:
n = 0
for pool in pools:
    pool_i = pools.index(pool)
    for entity in pool:
        e_index[entity].add(pool_i)
    n += 1
    if n % 1000 == 0:
        print(n, end=" ")

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 

In [39]:
cand_to_merging = list(e_index.values())
cand_to_merging = [list(el) for el in cand_to_merging if len(el) > 1]
print(len(cand_to_merging))

4672


In [40]:
print(cand_to_merging[:50])

[[4042, 11779], [3113, 9271], [4780, 1471], [41, 21], [2998, 287], [40178, 24487], [1817, 9258], [2288, 7905], [146, 3291], [2491, 1182], [1416, 13030], [691, 28], [1424, 10940], [5436, 726], [221, 3278], [1344, 177], [6547, 10300], [17496, 36760], [145, 108], [29923, 36075], [2491, 1182], [48921, 565], [5648, 6375], [2689, 926], [21336, 40148], [258, 1692], [747, 4927], [1568, 213], [299, 294], [5577, 13935], [11613, 4061], [557, 517], [1459, 7413], [1484, 6197], [557, 517], [954, 1413], [268, 7550], [22866, 7587], [464, 1576], [29346, 12159], [1592, 5549], [1628, 1822], [450, 7116], [11009, 1509], [3064, 13614], [2996, 1871], [29650, 15988], [17656, 4066], [10425, 1466], [557, 517]]


In [44]:
print(len(pools[41]))
print("********************************************************")
print(len(pools[21]))

33
********************************************************
35


In [32]:
# generate uris
entity_pages = {}
base = 'http://data.judaicalink.org/data/ep/'
i = 1000000
for pool in pools:
    entity_page_uri = base+str(i)
    entity_pages[entity_page_uri] = pool
    i += 1

In [33]:
tot_entities = [len(v) for v in entity_pages.values()]
tot_entities = sum(tot_entities)
print(tot_entities)

428307


In [34]:
inverted = {}
for ep_uri, pool in entity_pages.items():
    for entity_uri in pool:
        if entity_uri not in inverted:
            inverted[entity_uri] = [ep_uri]
        else:
            inverted[entity_uri].append(ep_uri)

In [35]:
for k,v in inverted.items():
    if len(v) > 1:
        print(k)
        print(v)
        print()

http://sws.geonames.org/3081368/
['http://data.judaicalink.org/data/ep/1000000', 'http://data.judaicalink.org/data/ep/1045257']

http://viaf.org/viaf/49228757
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

https://www.deutsche-digitale-bibliothek.de/entity/118578537
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

http://id.loc.gov/authorities/n79006935
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

http://www.wikidata.org/entity/Q9061
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

https://de.wikipedia.org/wiki/Karl_Marx
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

http://hub.culturegraph.org/entityfacts/118578537
['http://data.judaicalink.org/data/ep/1000006', 'http://data.judaicalink.org/data/ep/1000677']

http://www.historicum.net/themen/klas

http://www.wikidata.org/entity/Q319243
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

https://www.deutsche-digitale-bibliothek.de/entity/118777890
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

https://en.wikipedia.org/wiki/Alexandre_Koyr%C3%A9
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

http://id.loc.gov/authorities/n50044214
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

http://viaf.org/viaf/100191727
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

https://de.wikipedia.org/wiki/Alexandre_Koyr%C3%A9
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

http://hub.culturegraph.org/entityfacts/118777890
['http://data.judaicalink.org/data/ep/1000268', 'http://data.judaicalink.org/data/ep/1007550']

http:

http://d-nb.info/gnd/123212308
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

http://id.loc.gov/authorities/n78067477
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

http://www.wikidata.org/entity/Q534080
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

https://www.deutsche-digitale-bibliothek.de/entity/123212308
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

http://d-nb.info/gnd/123212308/about
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

http://hub.culturegraph.org/entityfacts/123212308
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

https://de.wikipedia.org/wiki/Jan_T._Gross
['http://data.judaicalink.org/data/ep/1000598', 'http://data.judaicalink.org/data/ep/1003301']

http://d-nb.info/gnd/118047


https://en.wikipedia.org/wiki/Meir_of_Rothenburg
['http://data.judaicalink.org/data/ep/1001182', 'http://data.judaicalink.org/data/ep/1002491']

http://d-nb.info/gnd/119136392/about
['http://data.judaicalink.org/data/ep/1001182', 'http://data.judaicalink.org/data/ep/1002491']

http://dbpedia.org/resource/Boris_Zakhoder
['http://data.judaicalink.org/data/ep/1001192', 'http://data.judaicalink.org/data/ep/1048522']

http://www.wikidata.org/entity/Q384859
['http://data.judaicalink.org/data/ep/1001254', 'http://data.judaicalink.org/data/ep/1010534']

https://en.wikipedia.org/wiki/Azriel_Hildesheimer
['http://data.judaicalink.org/data/ep/1001254', 'http://data.judaicalink.org/data/ep/1010534']

http://data.judaicalink.org/data/bhr/Hildesheimer_Esriel-1
['http://data.judaicalink.org/data/ep/1001254', 'http://data.judaicalink.org/data/ep/1010534']

http://steinheim-institut.de:50580/cgi-bin/bhr?id=718
['http://data.judaicalink.org/data/ep/1001254', 'http://data.judaicalink.org/data/ep/1010534


http://www.wikidata.org/entity/Q88935
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://d-nb.info/gnd/116468459/about
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://www.isni.org/0000000115705482
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://www.deutsche-biographie.de/pnd116468459.html#ndbcontent
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://kalliope-verbund.info/gnd/116468459
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://dbpedia.org/resource/David_Cassel
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

http://hub.culturegraph.org/entityfacts/116468459
['http://data.judaicalink.org/data/ep/1001871', 'http://data.judaicalink.org/data/ep/1002996']

https://de.wikiped

http://viaf.org/viaf/79065991
['http://data.judaicalink.org/data/ep/1004108', 'http://data.judaicalink.org/data/ep/1013142']

http://d-nb.info/gnd/11922982X
['http://data.judaicalink.org/data/ep/1004108', 'http://data.judaicalink.org/data/ep/1013142']

http://dbpedia.org/resource/Uri_Nissan_Gnessin
['http://data.judaicalink.org/data/ep/1004108', 'http://data.judaicalink.org/data/ep/1013142']

https://en.wikipedia.org/wiki/Uri_Nissan_Gnessin
['http://data.judaicalink.org/data/ep/1004108', 'http://data.judaicalink.org/data/ep/1013142']

http://catalogue.bnf.fr/ark:/12148/cb13604249g
['http://data.judaicalink.org/data/ep/1004109', 'http://data.judaicalink.org/data/ep/1004659']

http://id.loc.gov/authorities/n86862909
['http://data.judaicalink.org/data/ep/1004109', 'http://data.judaicalink.org/data/ep/1004659']

http://www.isni.org/0000000115115597
['http://data.judaicalink.org/data/ep/1004109', 'http://data.judaicalink.org/data/ep/1004659']

http://hub.culturegraph.org/entityfacts/1359390


http://viaf.org/viaf/10535470
['http://data.judaicalink.org/data/ep/1010077', 'http://data.judaicalink.org/data/ep/1033735']

http://www.wikidata.org/entity/Q6830314
['http://data.judaicalink.org/data/ep/1010102', 'http://data.judaicalink.org/data/ep/1033566']

http://viaf.org/viaf/14819483
['http://data.judaicalink.org/data/ep/1010102', 'http://data.judaicalink.org/data/ep/1033566']

http://viaf.org/viaf/49237109
['http://data.judaicalink.org/data/ep/1010136', 'http://data.judaicalink.org/data/ep/1033424']

http://www.wikidata.org/entity/Q7326646
['http://data.judaicalink.org/data/ep/1010136', 'http://data.judaicalink.org/data/ep/1033424']

http://www.wikidata.org/entity/Q5382261
['http://data.judaicalink.org/data/ep/1010226', 'http://data.judaicalink.org/data/ep/1033124']

http://viaf.org/viaf/55302034
['http://data.judaicalink.org/data/ep/1010226', 'http://data.judaicalink.org/data/ep/1033124']

http://d-nb.info/gnd/121094596
['http://data.judaicalink.org/data/ep/1010378', 'http://

['http://data.judaicalink.org/data/ep/1028343', 'http://data.judaicalink.org/data/ep/1029382']

http://hub.culturegraph.org/entityfacts/1017378673
['http://data.judaicalink.org/data/ep/1028343', 'http://data.judaicalink.org/data/ep/1029382']

http://d-nb.info/gnd/137682972
['http://data.judaicalink.org/data/ep/1028392', 'http://data.judaicalink.org/data/ep/1036044']

http://viaf.org/viaf/98460303
['http://data.judaicalink.org/data/ep/1028392', 'http://data.judaicalink.org/data/ep/1036044']

http://hub.culturegraph.org/entityfacts/137682972
['http://data.judaicalink.org/data/ep/1028392', 'http://data.judaicalink.org/data/ep/1036044']

http://www.wikidata.org/entity/Q6066038
['http://data.judaicalink.org/data/ep/1028499', 'http://data.judaicalink.org/data/ep/1033514']

http://viaf.org/viaf/7742208
['http://data.judaicalink.org/data/ep/1028602', 'http://data.judaicalink.org/data/ep/1033570']

http://d-nb.info/gnd/119183803
['http://data.judaicalink.org/data/ep/1028843', 'http://data.judai